In [1]:
!pip install pyarrow==14.0.1 requests==2.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 13.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2


In [2]:
!pip install transformers datasets==2.19.0 torch accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x8

In [22]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import GPT2Tokenizer
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

In [4]:
df=pd.read_csv('/content/customer.csv')

# Combine the columns into a single text field for training
df['input_text'] = df.apply(lambda row: f"Query: {row['Message']} Response: {row['Response']}", axis=1)

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df[['input_text']])

In [5]:
# Initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Set padding token
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [6]:
def tokenize_function(examples):
    return tokenizer(examples['input_text'], truncation=True,padding='max_length', max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/8100 [00:00<?, ? examples/s]

In [7]:
# Load GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set up data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [8]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    save_steps=10_000,
    save_total_limit=2,
)


In [9]:
# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)
trainer.train()

Step,Training Loss
500,1.310400


TrainOutput(global_step=759, training_loss=1.2230471082041543, metrics={'train_runtime': 3623.3744, 'train_samples_per_second': 6.706, 'train_steps_per_second': 0.209, 'total_flos': 6344170536960000.0, 'train_loss': 1.2230471082041543, 'epoch': 2.9970384995064165})

In [10]:

# Save the model
model.save_pretrained('./finetuned_gpt2')
tokenizer.save_pretrained('./finetuned_gpt2')


('./finetuned_gpt2/tokenizer_config.json',
 './finetuned_gpt2/special_tokens_map.json',
 './finetuned_gpt2/vocab.json',
 './finetuned_gpt2/merges.txt',
 './finetuned_gpt2/added_tokens.json')

In [11]:
# List the saved model files
import os
print(os.listdir('./finetuned_gpt2'))

['config.json', 'generation_config.json', 'tokenizer_config.json', 'model.safetensors', 'special_tokens_map.json', 'vocab.json', 'merges.txt']


In [18]:
from google.colab import files

# Compress the directory into a zip file
!zip -r finetuned_gpt2.zip ./finetuned_gpt2

# Download the zip file
files.download('finetuned_gpt2.zip')


updating: finetuned_gpt2/ (stored 0%)
updating: finetuned_gpt2/config.json (deflated 52%)
updating: finetuned_gpt2/generation_config.json (deflated 24%)
updating: finetuned_gpt2/tokenizer_config.json (deflated 55%)
updating: finetuned_gpt2/model.safetensors (deflated 7%)
updating: finetuned_gpt2/special_tokens_map.json (deflated 74%)
updating: finetuned_gpt2/vocab.json (deflated 68%)
updating: finetuned_gpt2/merges.txt (deflated 53%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained('./finetuned_gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('./finetuned_gpt2')

# Ensure pad_token is set to eos_token
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

def generate_response(input_text):
    # Tokenize input with padding and attention mask
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

    # Generate response with a high max_length
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],  # Include attention mask
        max_length=1024,  # Set to a high value to ensure complete responses
        num_return_sequences=1,
        temperature=0.5,  # Adjust temperature as needed
        top_k=50,  # Adjust top_k as needed
        top_p=0.95,  # Adjust top_p as needed
        no_repeat_ngram_size=2,  # Prevent repeating phrases
        early_stopping=True  # Stop early if end of sentence token is generated
    )

    # Decode and print the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(response)

# Example usage
generate_response("How can i know the cancellation charges")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


How can i know the cancellation charges? Response: i'll get right on it! i understand that you would like to know about the charges associated with canceling your subscription to our newsletter. to provide you with accurate information, could you please provide me with your email address or any other relevant details related to your cancellation? this will help me locate your account and provide the necessary details for you to cancel your newsletter subscription. thank you for your cooperation and patience. we appreciate your patience and cooperation in this matter. if you have any further questions or need further assistance, please don't hesitate to let me know. i'm here to help!"

i'm committed to ensuring that your request is processed promptly and efficiently. please let us know if there's anything else we can assist you in. your satisfaction is our top priority, and we're here every step of the way to ensure your convenience and satisfaction. let's work together to resolve this 